In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
from vllm import LLM

INFO 03-21 15:44:45 __init__.py:183] Automatically detected platform cuda.


In [4]:
import gc
import os

import torch


def clear():
    """Clear GPU memory"""
    visible_devices = os.getenv("CUDA_VISIBLE_DEVICES")
    print(f"Clearing GPU memory for {visible_devices}")
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
from datasets import load_dataset

ds = load_dataset('openai/gsm8k', 'main', split='train[:100]')
ds[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [ ]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

from verifiers.utils.cuda import get_half_precision_dtype


clear()

model = 'Qwen/QwQ-32B'
# model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

llm = LLM(
    model=model,
    dtype=get_half_precision_dtype(),
    gpu_memory_utilization=0.8,
    max_model_len=4096,
    trust_remote_code=True,
)


[2025-03-21 15:44:52,660] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: /usr/local/cuda-12.3/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: /usr/local/cuda-12.3/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_

Clearing GPU memory for 3
INFO 03-21 15:45:06 config.py:520] This model supports multiple tasks: {'score', 'classify', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 03-21 15:45:06 llm_engine.py:232] Initializing an LLM engine (v0.7.0) with config: model='Qwen/QwQ-32B', speculative_config=None, tokenizer='Qwen/QwQ-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/QwQ-32B, num_scheduler_steps=1, 

model-00001-of-00014.safetensors:   6%|6         | 304M/4.89G [00:00<?, ?B/s]

model-00006-of-00014.safetensors:   6%|6         | 294M/4.88G [00:00<?, ?B/s]

...

model-00010-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00011-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00012-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
example = ds[0]
prompt = example['question']
print(prompt)

: 

In [ ]:
def chat_completion(messages, max_tokens=512, temperature=0.5, top_p=0.95, **kwargs):
    """
    Perform chat completion with the given messages and token limit.

    Args:
        messages: List of message dictionaries with role and content
        max_tokens: Maximum tokens to generate in response

    Returns:
        Generated text from the model
    """
    sampling_params = SamplingParams(
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        **kwargs,
    )

    outputs = llm.chat(
        messages,
        sampling_params,
        continue_final_message=True,
        add_generation_prompt=False,
        use_tqdm=False,
    )
    return outputs[0].outputs[0].text

: 

In [ ]:
prefill = """
<think>
First, I need to determine how many clips Natalia sold in April. She sold 48 clips to her friends in April.

Next, in May, she sold half as many clips as she did in April. So, half of 48 is 24.

Finally, to find the total number of clips sold in both months, I'll add the April sales to the May sales: 48 plus 24 equals 72.
</think>

""".strip()

messages = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": prefill},
]

chat_completion(messages)

: 

In [ ]:
# compare baseline completion vs thinking budget completion by completion length and answer correctness
# Function to get baseline completion without thinking budget
def generate_baseline(prompt):
    messages = [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "<think>\n"},
    ]
    return chat_completion(messages)

: 

In [ ]:
example = ds[0]
prompt = example['question']
print(prompt)

: 

In [ ]:
response = generate_baseline(prompt)
print(response)


: 

In [ ]:
def generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=50):
    """
    Generate a response with controlled thinking budget.

    Args:
        prompt: The user prompt to respond to
        num_steps: Number of thinking steps to use
        tokens_per_step: Maximum tokens to generate per thinking step

    Returns:
        The final generated text with thinking steps and answer
    """
    # Start with the basic conversation
    messages = [
        {"role": "user", "content": prompt},
    ]

    # Initialize with the first thinking step
    initial_content = f"<think progress=1/{num_steps}>\n"
    assistant_msg = {"role": "assistant", "content": initial_content}
    messages.append(assistant_msg)

    # Track the complete response
    full_response = initial_content

    # Generate each thinking step iteratively
    for step in range(1, num_steps + 1):
        # Generate the next chunk of thinking
        step_response = chat_completion(messages, max_tokens=tokens_per_step)

        # Update our full response with this step's generation
        full_response += step_response

        # Check if thinking has naturally concluded
        if "</think>" in step_response:
            # If the model has finished thinking, we're done with the thinking steps
            print("Thinking completed naturally")
            break

        # Prepare for the next step if we're not at the end
        if step < num_steps:
            # Update the step number in the response content
            current_tag = f"<think progress={step}/{num_steps}>"
            next_tag = f"<think progress={step + 1}/{num_steps}>"
            updated_content = full_response.replace(current_tag, next_tag)

            # Update the assistant's message with the new content
            messages[-1]["content"] = updated_content
            full_response = updated_content

    # Check if thinking completed naturally
    if "</think>" not in full_response:
        # Force close the thinking tag if it wasn't closed
        full_response += "\n</think>\n"
        messages[-1]["content"] = full_response
    print(full_response)
    # Generate the final answer with larger token limit
    final_answer = chat_completion(messages, max_tokens=512)
    full_response += final_answer

    return full_response


: 

In [ ]:
response = generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=10)
print(response)


: 

In [ ]:
# Get completions for multiple examples
num_examples = 10
results = []

for i in range(num_examples):
    example = ds[i]
    prompt = example['question']
    
    # Generate both types of completions
    baseline = generate_baseline(prompt)
    thinking = generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=10)
    
    # Compare lengths
    baseline_length = len(baseline.split())
    thinking_length = len(thinking.split())
    
    # Extract final answers (assuming they end with #### number format)
    baseline_answer = baseline.split('####')[-1].strip() if '####' in baseline else None
    thinking_answer = thinking.split('####')[-1].strip() if '####' in thinking else None
    
    # Compare with ground truth
    correct_answer = example['answer'].split('####')[-1].strip()
    baseline_correct = baseline_answer == correct_answer if baseline_answer else False
    thinking_correct = thinking_answer == correct_answer if thinking_answer else False
    
    results.append({
        'baseline_length': baseline_length,
        'thinking_length': thinking_length,
        'baseline_correct': baseline_correct,
        'thinking_correct': thinking_correct
    })

# Calculate aggregate statistics
avg_baseline_length = sum(r['baseline_length'] for r in results) / len(results)
avg_thinking_length = sum(r['thinking_length'] for r in results) / len(results)
baseline_accuracy = sum(1 for r in results if r['baseline_correct']) / len(results)
thinking_accuracy = sum(1 for r in results if r['thinking_correct']) / len(results)

print(f"Average completion lengths:")
print(f"Baseline: {avg_baseline_length:.1f} words")
print(f"Thinking Budget: {avg_thinking_length:.1f} words")
print(f"\nAnswer accuracy:")
print(f"Baseline: {baseline_accuracy:.1%}")
print(f"Thinking Budget: {thinking_accuracy:.1%}")



: 

: 